In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/lin

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-19 04:47:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-19 04:47:28 (6.61 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date","yyyy-MM-dd").alias("review_date")]).drop_duplicates()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTWZWZMEWNGVI|   11758377|B00LK5J73U|     449125124| 2015-08-31|
|R1KLL9E7663ZQ0|   15159309|B00JZZ554C|     241730228| 2015-08-31|
|R28AGFK7RO4RQ9|   21635813|B00TQ84AEE|     816534175| 2015-08-31|
| R533H9P9FP9DV|   42626174|B00ONXWMNS|     195625988| 2015-08-31|
|R1K640V6LRG2BF|   19619373|B00PTL3SX6|     455995958| 2015-08-31|
|R3SII9NOMIB9J7|   20241560|B004QZBEFK|     327023258| 2015-08-31|
| RZSGB6M7T051M|   14386248|B007HKOL92|     378583797| 2015-08-31|
|R2CHS8M3JTXUZK|   28093946|B007Y0862I|     598324482| 2015-08-31|
| RSIU35IQBMUSV|   12825257|B008KIQ6MK|     302631006| 2015-08-31|
| R2UC0X9GNSP5Q|   49955665|B00F667QS2|     336944964| 2015-08-31|
|R2YQ6CD5TCFWAM|    3643455|B000LSL5ZK|      52088970| 2015-08-31|
|R1KEY6N21TT00E|     164754|B00H8MSEWY|     261463126| 2015-08

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             2|
|   44524872|             1|
|   16711087|             1|
|   14116630|             3|
|   10548283|             1|
|   12879980|            15|
|   11913481|             1|
|   49875798|             3|
|   40993003|             1|
|    6272214|             3|
|     417296|             1|
|   36524024|             2|
|   16146148|             1|
|   43274367|             1|
|   38692392|            14|
|   24359722|             1|
|   45450058|             5|
|    1960118|             1|
|   36980552|             1|
|   10200363|             2|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00Y8QMQN6|Best Baby Teether...|
|B002WN2BY6|The First Years T...|
|B000AV7O4O|Boon Frog Pod Bat...|
|B00UFOBDNY|Kinsa Smart Stick...|
|B00XF7DZ34|Critter Piller Ki...|
|B00WKIR22E|Poly Comfortable ...|
|B001EU35BQ|Peg Perego Tatami...|
|B001TWI81G|Thermos Reusable ...|
|B007EJ7J4U|Kaloo Doudou Beig...|
|B00M8MGGQA|Large Baby Bag Or...|
|B00Y2CJWKQ|Clinical Digital ...|
|B00Y8ION80|★ Baby Pacifier &...|
|B00UJOWZ96|Corner Cushion Ba...|
|B00DZ3K64Y|Sassy Soft Grip N...|
|B00JBYPKE2|Maxi Cosi Pria 85...|
|B00HK6GWTU|Ju-Ju-Be Be Light...|
|B00IYU4O8W|   Lamaze Cloth Book|
|B00SCDM4FQ|Olababy Silicone ...|
|B004UOTWJ2|Charlie Banana - ...|
|B001LF3YQK|Skip Hop Baby Swi...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542| 2015-08-31|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401| 2015-08-31|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336| 2015-08-31|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304| 2015-08-31|
| REV51EW323H8W|    9287389|B010UX9T5I|     446691106| 2015-08-31|
|R2GQ3W03WIUZKE|   32840762|B00VWBY7SC|     271204734| 2015-08

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
|R2JWY4YRQD4FOP|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
|R13EPSFP5DODN5|          4|            0|          0|   N|
| R6RBP4HTE67SY|          5|            0|          0|   N|
|R15B3EU40RSU2W|          5|            0|          0|   N|
| RP4DD53A4ZJA2|          5|            0|          0|   N|
|R2C99DJEO4RZ4K|          5|            3|          4|   N|
| REV51EW323H8W|          5|            0|          0|   N|
|R2GQ3W03WIUZKE|          5|            0|          0|   N|
| RTI1YI7K6GE3D|          5|            0|          0|   N|
|R3V9C2C0SPSZU6|          5|            

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-challenge.cn0kp2wnsog3.us-east-2.rds.amazonaws.com:5432/Mod_16_Challenge_DB"
config = {"user":"postgres", 
          "password": "12345678", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [14]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [21]:
# Write review_id_df to table in RDS
try:
    review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)
except:
  None